In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
#import cvxpy
import math
import numpy as np
#import cubic_spline_planner

N_IND_SEARCH = 10  # Search index number
show_animation = True

In [2]:
import matplotlib
matplotlib.__version__

'3.1.0'

!rm -f Data_Processor.pyc
Remove=True
if Remove==True:
    !rm -r Fig_iLQR
    !rm -r Rec_iLQR
    !rm -r Rec_iLQR_Raw

In [3]:
import numpy as np
import math
from copy import copy, deepcopy
TARGET_SPEED = 20# / 3.6  # [m/s] target speed
MAX_TIME=1500
T=5
NX=4
NU=2
DT=0.2
WB=50
MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 25#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -25#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 5.0  # maximum accel [m/ss]
    


In [4]:
from Motion import Motion,State
R = np.diag([0.5, 10.0])  # input cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix


def np_quad_form(a,b):
        res=a @ b @ a.T
        return res
    
    
    
def sim_n_steps(_x0,U,xref,N):
    _U=U.copy()
    X=np.zeros((NX,N+1),dtype=np.float64)
    X[:,0]=_x0
    
    total_grad=0
    loss=0
    for t in range(N):
        x_t=X[:,t]
        u_t=_U[:,t]
        xref_t=xref[:,t]
        x_t1=sim_1_step(x_t,u_t)
        loss += np_quad_form(u_t, R) + np_quad_form(xref_t - x_t, Q)
        #cc=calc_cost(x_t1,u_t,xref_t,t)
        X[:,t+1]=x_t1.copy()
    return X,loss,U

def sim_1_step(_X,_U):
    
    _state=State(x=_X[0],y=_X[1],v=_X[2],yaw=_X[3])
    nst,Const=Motion.get_next_state(_state,_U[0],_U[1])
    x_t1=nst.array()
    return x_t1#np.array(x_t1,dtype=np.float64)

In [5]:
class iLQRController():
    def __init__(self):
        self.STATE_SIZE=NX
        self.INPUT_SIZE=NU
        self.max_iter=10
        self.dt=0.2
        self.lamb_factor=1.0
        self.lamb_max=10e4
        self.tN=T
        self.eps_converge=0.001
        
    def init_forward_matrix(self):
        self.f_x = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # df / dx
        self.f_u = np.zeros((self.tN, self.STATE_SIZE, self.INPUT_SIZE)) # df / du
        
    def init_cost(self):
        self.l = np.zeros((self.tN,1)) # immediate state cost 
        self.l_x = np.zeros((self.tN, self.STATE_SIZE)) # dl / dx
        self.l_xx = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # d^2 l / dx^2
        self.l_u = np.zeros((self.tN, self.INPUT_SIZE)) # dl / du
        self.l_uu = np.zeros((self.tN, self.INPUT_SIZE, self.INPUT_SIZE)) # d^2 l / du^2
        self.l_ux = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # d^2 l / du / dx
    
    def calc_cost(self, _x, _u, _xref,t):
        
        
        self.l[t] = np_quad_form(_u, R) + np_quad_form(_xref - _x, Q)
        
        loss_x1=2*Q[0,0]*(_xref[0]-_x[0])*(-1)
        loss_x2=2*Q[1,1]*(_xref[1]-_x[1])*(-1)
        loss_x3=2*Q[2,2]*(_xref[2]-_x[2])*(-1)
        loss_x4=2*Q[3,3]*(_xref[3]-_x[3])*(-1)
            
        self.l_x[t]=np.array([loss_x1,loss_x2,loss_x3,loss_x4])
        self.l_xx[t]=2 * Q
        
        Qdiag=np.array([Q[0,0], Q[1,1], Q[2,2], Q[3,3]],dtype=np.float64)
        error=np.array(_xref-_x)
        
        loss_u1 = 2. * _u[0] * R[0,0] #+ np.sum(2*Qdiag*error*(-1)*_B[:,0])
        loss_u2 = 2. * _u[1] * R[1,1] #+ np.sum(2*Qdiag*error*(-1)*_B[:,1])
        
        self.l_u[t] = np.array([loss_u1, loss_u2])
        self.l_uu[t] = 2. * R
        self.l_ux[t] = np.zeros((NU, NX))
    
    def init_ks(self):
        self.k = np.zeros((self.tN, self.INPUT_SIZE)) # feedforward modification
        self.K = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # feedback gain

  
    def calc_forward_matrix(self,_X,_U,t):
        A, B, C = Motion.get_linear_model_matrix(_X[2],_X[3], _U[1])
        self.f_x[t] = A
        self.f_u[t] = B 
        
    def ilqr(self, x0, U,xref): 
        
        lamb = 1.0 # regularization parameter
        sim_new_trajectory = True
        tN = U.shape[0] # number of time steps
        X=np.zeros((NX,T+1),dtype=np.float64)
        X[:,0]=x0.copy()
        for ii in range(self.max_iter):

            if sim_new_trajectory == True: 
                #X, cost, _ = sim_n_steps(x0, U,xref,T)
                #oldcost = copy(cost) # copy for exit condition check

                
                self.init_forward_matrix()
                self.init_cost()
                for t in range(self.tN):
                    x_t=X[:,t].copy()
                    u_t=U[:,t].copy()
                    xref_t=xref[:,t].copy()
                    self.calc_forward_matrix(x_t,u_t,t)
                    self.calc_cost(x_t, u_t,xref_t,t)
                    X[:,t+1]=sim_1_step(x_t,u_t)
                _cost =np.sum(self.l)
                sim_new_trajectory = False

            V = self.l[-1].copy()  
            V_x = self.l_x[-1].copy() 
            V_xx = self.l_xx[-1].copy() 
            self.init_ks()
 
            for t in range(self.tN-2, -1, -1):
                
                Q_x = self.l_x[t] + np.dot(self.f_x[t].T, V_x) 
                Q_u = self.l_u[t] + np.dot(self.f_u[t].T, V_x)

                Q_xx = self.l_xx[t] + np.dot(self.f_x[t].T, np.dot(V_xx, self.f_x[t])) 
                Q_ux = self.l_ux[t] + np.dot(self.f_u[t].T, np.dot(V_xx, self.f_x[t]))
                Q_uu = self.l_uu[t] + np.dot(self.f_u[t].T, np.dot(V_xx, self.f_u[t]))

                Q_uu_evals, Q_uu_evecs = np.linalg.eig(Q_uu)
                Q_uu_evals[Q_uu_evals < 0] = 0.0
                #Q_uu_evals += lamb
                Q_uu_inv = np.dot(Q_uu_evecs, np.dot(np.diag(1.0/Q_uu_evals), Q_uu_evecs.T))
                #k[t]=-1*l_u[t]
                self.k[t] = -1. * np.dot(Q_uu_inv, Q_u)
                self.K[t] = -1. * np.dot(Q_uu_inv, Q_ux)

                V_x = Q_x - np.dot(self.K[t].T, np.dot(Q_uu, self.k[t]))
                V_xx = Q_xx - np.dot(self.K[t].T, np.dot(Q_uu, self.K[t]))

            U_new = np.zeros((NU,T),dtype=np.float64)
            x_new = np.zeros((NX,T),dtype=np.float64)
            x_new = x0.copy()
            for t in range(self.tN - 1): 
               
                U_new[:,t] = U[:,t] + self.k[t] + np.dot(self.K[t], x_new - X[:,t])
                x_new_1= sim_1_step(x_new, U_new[:,t])
                x_new=x_new_1.copy()
            # evaluate the new trajectory 
            X_new, _new_cost , _= sim_n_steps(x0, U_new,xref,T)
            #if 1==1:
      
            if _new_cost < _cost: 
                # decrease lambda --> get closer to Newton's method
                lamb /= self.lamb_factor

                X = copy(X_new) 
                U = copy(U_new) 
                #oldcost = copy(cost)
                #cost = copy(cost_new)

                sim_new_trajectory = True 

                #print('...',_cost,_new_cost)
                if ii > 0 and ((abs(_cost-_new_cost)/_new_cost) < self.eps_converge):
                    print("break 1",ii)
                    break

            else: 
                # increase lambda --> closer to gradient descent
                lamb *= self.lamb_factor
                if lamb > self.lamb_max: 
                    print("break 2",ii)
                    break
        print('return ii',ii,_new_cost,_cost)
        return X, U, _cost
    

In [6]:
import os
from Airsim_Env import Car_Env
from Data_Processor import Data_Processor
from Car_Plot import Move_Contour,Draw_Car

In [7]:
#!rm -r Exp2_456

In [8]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
rec=True
max_len=500
qlen=15
car_move=True
seed=123
root='AS4s_CT_'+str(seed)+'/'
if not os.path.exists(root):
        os.mkdir(root)
        
np.random.seed(seed)
Env=Car_Env(rec=rec,max_len=max_len,qlen=qlen,path=root,loop='2',seed=seed)
[init_state,map_points]=Env.reset()
mpx=np.array(map_points[:max_len,0])#-ofx
mpy=np.array(map_points[:max_len,1])#-ofy
ix,iy,iyaw,ivel=init_state[0]
_traj=init_state[1][0]
_dind=init_state[1][1]
_end=init_state[1][2]
ofx,ofy,ofang=init_state[2]
imgs=init_state[3]
state = State(x=ix, y=iy, yaw=iyaw, v=ivel)
#print(init_state)

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [9]:
if rec==True:
    if not os.path.exists(root+'Rec_iLQR_f/'):
        os.mkdir(root+'Rec_iLQR_f')
    if not os.path.exists(root+'Rec_iLQR_l/'):
        os.mkdir(root+'Rec_iLQR_l')
    if not os.path.exists(root+'Rec_iLQR_r/'):
        os.mkdir(root+'Rec_iLQR_r')
    if not os.path.exists(root+'Rec_iLQR_drv/'):
        os.mkdir(root+'Rec_iLQR_drv')
    
    if not os.path.exists(root+'Rec_iLQR_bc/'):
        os.mkdir(root+'Rec_iLQR_bc')
    if not os.path.exists(root+'Fig_iLQR/'):
        os.mkdir(root+'Fig_iLQR')


In [10]:
import cv2
import matplotlib.gridspec as gridspec
import time

phi=0
rot_mat=np.array([[math.cos(phi),math.sin(phi)],[-math.sin(phi),math.cos(phi)]])
dis_time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]

log_state=[]
log_traj=[]
log_img=[]
log_act=[]
log_next_state=[]
log_reward=[]


#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
_index=0
#cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''


_Data_Processor=Data_Processor(_speed=TARGET_SPEED,tlen=qlen)
acc_st=State(x=0,y=0)
itr=0
#fig,axes=plt.subplots(nrows=2,ncols=1,figsize=(10,10))
fig=plt.figure(figsize=(10,10))
gs = fig.add_gridspec(3, 1)
ax1=fig.add_subplot(gs[0,:])
ax2=fig.add_subplot(gs[1:,:])
axes=[ax1,ax2]


def map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _ofang=-1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)

    _vect[0]+=_ofx
    _vect[1]+=_ofy
    return [_vect[0],_vect[1]]
def inv_map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _vect[0]-=_ofx
    _vect[1]-=_ofy
    _ofang=1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)
    return [_vect[0],_vect[1]]
U=np.zeros((NU,T),dtype=np.float64)
iLQR=iLQRController()
init_time=time.time()
while MAX_TIME >= dis_time:
       
        _full_state = np.array([state.x, state.y, state.v, state.yaw],dtype=np.float64)
    
        
        _full_state_traj=_Data_Processor.update_trajectory(_traj)
        xref, dref = _Data_Processor.get_ref_trajectory(state,dl,DT,T)
        
        if _end==True:
            break

        _index+=_dind
        
        #oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(xref, Motion_Status, dref, oa, odelta)
        X,U, cost= iLQR.ilqr(_full_state, U, xref )
        oa=U[0,1]
        odelta=U[1,1]
        ox=X[0,1]
        oy=X[1,1]
        ov=X[2,1]
        oyaw=X[3,1]
        state_str=' x: '+str('%0.4f'%state.x)+' y: '+str('%0.4f'%state.y)+' v: '+str('%0.4f'%state.v)+' yaw: '+str('%0.4f'%np.rad2deg(state.yaw))+'\n'
        xref_str =' x: '+str('%0.4f'%xref[0][0])+' y: '+str('%0.4f'%xref[1][0])+' v: '+str('%0.4f'%xref[2][0])+' yaw: '+str('%0.4f'%np.rad2deg(xref[3][0]))+'\n'
        time_str ='Time(Sec): '+str('%0.4f'%dis_time)+' Sec Distance(m): '+str('%0.4f'%_index)+' m '+'\n'
        #xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        act_str  =' accel: '+str('%0.4f'%accl)+' gas: '+str('%0.4f'%gas)+' steer: '+str('%0.4f'%np.rad2deg(steer))+'\n'
    
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        max_steer=np.deg2rad(30)
        #ai=1-state.yaw*1.2
        max_accl=5
        if odelta is not None:
            if abs(U[1,0])<max_steer:
                  di= U[1,0]
            else:
                di=(U[1,0]/abs(U[1,0]))*max_steer
            
            if abs(U[0,0])<max_accl:
                  ai=U[0,0]
            else:
                ai=(U[0,0]/abs(U[0,0]))*max_accl
        #accl=max_accl/16.0 - abs(ofang)/100.0
        accl=max_accl/16.0 - abs(0)/10.0
        steer=di
        #if state.yaw>0.6:
        #      brake=0.9
        #else:
        brake=0.0#abs(ofang)/25
        
        #act_str=str(steer)+'\n'
        _log_state=[ofx,ofy,_full_state[2],ofang]
        nst,path,offset,imgs,rw,ter,info=Env.step([steer,accl,brake])
        imgf=imgs[0]
        #imgl=imgs[1]
        #imgr=imgs[2]
        #imgdrv=imgs[3]
        #imgbc=imgs[4]
        
        
        simgf=imgs[1]
        '''
        simgl=imgs[6]
        simgr=imgs[7]
        simgdrv=imgs[8]
        simgbc=imgs[9]
        
        dimgf=imgs[10]
        dimgl=imgs[11]
        dimgr=imgs[12]
        dimgdrv=imgs[13]
        dimgbc=imgs[14]#Env.render(mode='rgb_array')
        '''
        _traj,_dind,_end=path
        ofx,ofy,ofang=offset
        
        gas=Env.gas
        
        state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        _log_next_state=[ofx,ofy,state.v,ofang]
        #Motion_Status.set_state(next_state)
        curr_time=time.time()-init_time
        log_state.append(_log_state)
        log_traj.append(_full_state_traj)
        log_img.append(imgs)
        log_act.append([accl,steer,brake])
        log_next_state.append(_log_next_state)
        log_reward.append([dis_time,curr_time,_index,rw,ter])
        
        dis_time = dis_time + DT
        if car_move==True:
            _xref=xref.copy()
            [_ox,_oy]=map_plot([ox,oy],ofx,ofy,ofang)
            [_xref[0,:],_xref[1,:]]=map_plot([xref[0,:],xref[1,:]],ofx,ofy,ofang)
            [_x,_y]=map_plot([state.x,state.y],ofx,ofy,ofang)
            car_x,car_y,car_ang=ofx,ofy,ofang
            _mpx,_mpy=mpx,mpy
        else:
            _ox,_oy=ox,oy
            _xref=xref
            _x,_y=state.x,state.y
            car_x,car_y,car_ang=state.x,state.y,state.yaw
            [_mpx,_mpy]=inv_map_plot([mpx,mpy],ofx,ofy,ofang)
        x.append(_x)
        y.append(_y)
        yaw.append(ofang)
        v.append(state.v)
        t.append(dis_time)
        d.append(di)
        a.append(ai)

        
    
      
        
        if show_animation:  # pragma: no cover
            axes[0].cla()
            axes[1].cla()
            axes[0].text(.01,.6,'Running : '+time_str,fontsize=10,color='c')
            axes[0].text(.01,.4,'state : '+state_str,fontsize=10,color='r')
            axes[0].text(.01,.2,'xref : '+xref_str,fontsize=10,color='b')
            axes[0].text(.01,0,'act : '+act_str,fontsize=10,color='g')
            
            if ox is not None:
                axes[1].plot(_ox, _oy, "xr", label="MPC",alpha=1.0)
            axes[1].plot(_mpx, _mpy, "-b", label="course")
            axes[1].plot(x, y, "oc", label="trajectory")
            axes[1].plot(_xref[0, :], _xref[1, :], "xk", label="xref",alpha=1.0)
            axes[1].plot(_mpx[_index], _mpy[_index], "xg", label="target",alpha=1.0)
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            car_points=Draw_Car(car_x,car_y)
            rp=Move_Contour(car_points,0,0,car_ang)
            r_car_points=rp.apply()
            r_points=np.array(r_car_points,dtype=np.float64)
            axes[1].plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            #plt.title("Time[s]:" + str(round(time, 2))
            #          + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
            #          + '\nyaw'+str(state.yaw)
            #          + '\nxref_yaw'+str(xref[3][0])
            #          + '\nsteer'+str(steer))
            #fig.canvas.draw()
            plt.pause(0.0001)
            if rec==True:
                if itr%1==0:
                    plt.savefig(root+'Fig_iLQR/'+str(int(itr/1))+'.jpg')
                    cv2.imwrite(root+'Rec_iLQR_f/'+str(int(itr/1))+'.jpg',imgf)
                    #cv2.imwrite(root+'Rec_iLQR_l/'+str(int(itr/1))+'.jpg',imgl)
                    #cv2.imwrite(root+'Rec_iLQR_r/'+str(int(itr/1))+'.jpg',imgr)
                    #cv2.imwrite(root+'Rec_iLQR_drv/'+str(int(itr/1))+'.jpg',imgdrv)
                    #cv2.imwrite(root+'Rec_iLQR_bc/'+str(int(itr/1))+'.jpg',imgbc)
                    
                    cv2.imwrite(root+'Rec_iLQR_f/s_'+str(int(itr/1))+'.jpg',simgf)
                    '''
                    cv2.imwrite(root+'Rec_iLQR_l/s_'+str(int(itr/1))+'.jpg',simgl)
                    cv2.imwrite(root+'Rec_iLQR_r/s_'+str(int(itr/1))+'.jpg',simgr)
                    cv2.imwrite(root+'Rec_iLQR_drv/s_'+str(int(itr/1))+'.jpg',simgdrv)
                    cv2.imwrite(root+'Rec_iLQR_bc/s_'+str(int(itr/1))+'.jpg',simgbc)
                    
                    cv2.imwrite(root+'Rec_iLQR_f/d_'+str(int(itr/1))+'.jpg',dimgf)
                    cv2.imwrite(root+'Rec_iLQR_l/d_'+str(int(itr/1))+'.jpg',dimgl)
                    cv2.imwrite(root+'Rec_iLQR_r/d_'+str(int(itr/1))+'.jpg',dimgr)
                    cv2.imwrite(root+'Rec_iLQR_drv/d_'+str(int(itr/1))+'.jpg',dimgdrv)
                    cv2.imwrite(root+'Rec_iLQR_bc/d_'+str(int(itr/1))+'.jpg',dimgbc)
                    '''
           
            itr+=1
            if ter==True:
                print('terminal ... ')
                
if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(car_x, car_y, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


break 1 1
return ii 1 1655.31566861 1655.31972609
break 1 1
return ii 1 1655.31970008 1655.31970082
break 1 1
return ii 1 1655.31990346 1655.31990346
return ii 9 1655.31981202 1655.31981202
break 1 1
return ii 1 1514.51972893 1514.51994352
break 1 1
return ii 1 1429.80822792 1429.8082579
break 1 1
return ii 1 1357.19053402 1357.19054197
break 1 1
return ii 1 1272.20154664 1272.20155056
break 1 1
return ii 1 1173.58486998 1173.58487429
break 1 1
return ii 1 1083.48980347 1083.48980889
break 1 1
return ii 1 1009.2675321 1009.26753821
break 1 1
return ii 1 925.613491962 925.613501294
break 1 1
return ii 1 854.660113183 854.660120234
break 1 1
return ii 1 799.138658741 799.138663002
break 1 1
return ii 1 755.859866065 755.859867857
break 1 1
return ii 1 712.82286319 712.822866504
break 1 1
return ii 1 695.707824257 695.70782542
break 1 1
return ii 1 667.881999283 667.88200271
break 1 1
return ii 1 659.246231449 659.246281851
break 1 1
return ii 1 652.360366036 652.360387762
break 1 1
retur

break 1 1
return ii 1 627.437177476 627.437211209
break 1 1
return ii 1 627.671608472 627.671610321
break 1 1
return ii 1 628.681663522 628.681666349
break 1 1
return ii 1 630.220391209 630.220393194
break 1 1
return ii 1 631.37346422 631.373464604
break 1 1
return ii 1 632.515620423 632.515620942
break 1 1
return ii 1 634.266928118 634.266929521
break 1 1
return ii 1 628.516899676 628.516970433
break 1 1
return ii 1 628.808445092 628.808447089
break 1 1
return ii 1 629.346680485 629.346682522
break 1 1
return ii 1 630.186120925 630.186122749
break 1 1
return ii 1 631.136665344 631.136666647
break 1 1
return ii 1 634.130210137 634.130224248
break 1 1
return ii 1 637.601984303 637.601992403
break 1 1
return ii 1 643.132094766 643.132105382
break 1 1
return ii 1 632.638557159 632.638625668
break 1 1
return ii 1 637.146994678 637.147013174
break 1 1
return ii 1 644.481996005 644.482019211
break 1 1
return ii 1 633.543366509 633.543482702
break 1 1
return ii 1 637.294647029 637.29465992
br

In [11]:
log_dict={
'state':log_state,
'traj':log_traj,
'image':log_img,
'action':log_act,
'next_sate':log_next_state,
'time_reward':log_reward
}

In [12]:
import pickle
f=open(root+'dataset_iLQR','wb')
pickle.dump(log_dict,f)
f.close()
itr

261

In [13]:
import pickle
root='AS8_123/'
f=open(root+'dataset_iLQR','rb')
dataset=pickle.load(f)
f.close()

In [14]:
from matplotlib import pyplot as plt
import numpy as np
st=np.array(dataset['action'])[:,:]
plt.plot(st)
plt.show()

In [15]:
st=dataset['state'][:,0]
st

TypeError: list indices must be integers or slices, not tuple

In [ ]:
traj=np.array(log_traj)#np.array(dataset['traj'])
plt.plot(traj[])
plt.show()

In [ ]:
traj[200]

In [ ]:
conda install numpy, scipy 
pip install matplotlib , opencv-python , pillow
pip install mshpack-rpc-python

In [ ]:
car_state=client.getCarState()

In [ ]:
car_state

In [ ]:
import numpy as np
import  math
def quaternion_to_euler(x, y, z, w):

        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll = math.atan2(t0, t1)
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch = math.asin(t2)
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw = math.atan2(t3, t4)
        return [yaw, pitch, roll]
quaternion_to_euler(0,0,-0.7040445804595947, 0.7101558446884155),

In [ ]:
car_state